In [1]:
from openai import OpenAI

client = OpenAI(
    # #将这里换成你在aiproxy api keys拿到的密钥
    api_key="fk204054-M7aFv2t5pdWOSBMTURsUClih3sPHb8qQ|ck872-e5b3a2d",
    # 这里将官方的接口访问地址，替换成aiproxy的入口地址
    base_url="https://openai.api2d.net/v1"
)

def get_completion(prompt, stream=False, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages = messages,
        model = model,
        stream = stream,
        response_format={ "type": "json_object" },
    )
    # 如果是连续输出模式
    if stream==True:
        # 返回一个生成器
        return response
    else:
        # 一次性输出模式
        return response.choices[0].message.content

In [2]:
import numpy as np
import pandas as pd
import json
data = pd.read_csv('用户评论.csv', header=None)
data.head()

,0
0,安装师傅虽然有黑点，但最终还是解决了问题，感谢他们的努力。
1,物流真的很迅速，让人惊喜。
2,安全阀性能很稳定，非常安心，好评！
3,喷头的质量很不错，一切正常，满意度很高。
4,价格太高了，200的安装费真的令人不满意。


In [3]:
# 用于存储每个批次的字符串表示
batch_strings = []

# 使用 range 和 iloc 循环处理每 10 行
for i in range(0, len(data), 10):
    batch = data.iloc[i:i+10]
#     print(batch)  # 处理这 10 行数据
    # 将每行数据转换成字符串，每个句子为一行
    batch_str = '\n'.join(batch[0].astype(str))

    # 将批次字符串添加到列表中
    batch_strings.append(batch_str)

In [4]:
# 输出或处理每个批次的字符串
for batch_str in batch_strings:
    print(batch_str)
    print("-"*20)  # 批次之间的分隔符

安装师傅虽然有黑点，但最终还是解决了问题，感谢他们的努力。
物流真的很迅速，让人惊喜。
安全阀性能很稳定，非常安心，好评！
喷头的质量很不错，一切正常，满意度很高。
价格太高了，200的安装费真的令人不满意。
与我期望的差距太大，额外购买了不少配件，感觉有点浪费。
尽管有些配件需要额外购买，但整体体验还是很好的。
购买时应该提供更多的建议，我买小了，现在感到后悔。
虽然买小了，售后团队还是提供了一些帮助，感到满意。
感觉售后团队对问题处理不够积极，有待改进。
--------------------
价格在购买后突然降价，感觉被坑了一把，不开心。
售后服务真的需要改进，更加贴心一些。
喷头漏水的问题让我感到非常恼火，需要尽快解决。
不太满意产品的性能表现，期望更高一些。
价格太高了，装好后的效果非常值得。
购买后发现一些质量问题，让我有点失望。
物流虽然快，但包装不够严实，导致产品有轻微损坏。
安全阀出现了一些问题，让我有点担心。
服务电话投诉后降价，但仍觉得不太公平，不满意。
买完降价了，打了400的服务电话投诉后降价到260，真心黑，太恶心了。
--------------------
安全阀在使用时偶尔会出现一些问题，需要进一步改进。
物流速度非常快，令人惊讶，服务周到。
安装师傅虽然遇到了一些挑战，但最终还是解决了问题，感到满意。
产品的性能稳定，一直以来都表现得很可靠。
价格略高，但总体来说物有所值。
购买时需要更多的信息和建议，以避免后悔。
售后服务表现一般，有时响应不及时。
配件质量较差，导致使用不够顺畅。
喷头设计不够人性化，容易发生漏水问题。
安全阀性能不错，没有出现任何问题，非常满意。
--------------------
价格适中，不算太贵，但也不算便宜。
装配过程相对顺利，没有太多问题。
售后团队提供了很好的支持，解决了我的问题。
产品包装得很好，没有任何损坏。
总体来说，购物体验非常令人愉快。
安全阀的外观设计不够吸引人，可以更精致一些。
价格合理，符合市场水平，没有太大问题。
安装师傅服务态度良好，非常专业。
使用过程中没有出现任何问题，非常满意。
物流速度快，但配送过程中出现了一些小问题，需要改进。
--------------------


In [5]:
# 创建一个空的 DataFrame 用于存放所有评论
all_reviews_df = pd.DataFrame(columns=['评论', '评价'])

# 使用 range 和 iloc 循环处理每 10 行
for i in range(0, len(data), 10):
    batch = data.iloc[i:i+10]
#     print(batch)  # 处理这 10 行数据
    # 将每行数据转换成字符串，每个句子为一行
    batch_str = '\n'.join(batch[0].astype(str))
    # 得到JSON格式的数据
    json_data = get_completion(prompt = f"请对以下用户评论进行分类，得到‘好评’或‘差评’的结果。把结果用JSON形式输出，以用户评论为键，评论结果为值。用户评论：\n{batch_str}")
    print(json_data)
    # 字符串转字典格式
    json_data = json.loads(json_data)
    # 将当前的 JSON 对象转换为 DataFrame
    df = pd.DataFrame(list(json_data.items()), columns=['评论', '评价'])
    # 将转换后的 DataFrame 附加到主 DataFrame 中
    all_reviews_df = pd.concat([all_reviews_df, df], ignore_index=True)

{
    "安装师傅虽然有黑点，但最终还是解决了问题，感谢他们的努力。": "好评",
    "物流真的很迅速，让人惊喜。": "好评",
    "安全阀性能很稳定，非常安心，好评！": "好评",
    "喷头的质量很不错，一切正常，满意度很高。": "好评",
    "价格太高了，200的安装费真的令人不满意。": "差评",
    "与我期望的差距太大，额外购买了不少配件，感觉有点浪费。": "差评",
    "尽管有些配件需要额外购买，但整体体验还是很好的。": "好评",
    "购买时应该提供更多的建议，我买小了，现在感到后悔。": "差评",
    "虽然买小了，售后团队还是提供了一些帮助，感到满意。": "好评",
    "感觉售后团队对问题处理不够积极，有待改进。": "差评"
}
{
    "价格在购买后突然降价，感觉被坑了一把，不开心。": "差评",
    "售后服务真的需要改进，更加贴心一些。": "差评",
    "喷头漏水的问题让我感到非常恼火，需要尽快解决。": "差评",
    "不太满意产品的性能表现，期望更高一些。": "差评",
    "价格太高了，装好后的效果非常值得。": "好评",
    "购买后发现一些质量问题，让我有点失望。": "差评",
    "物流虽然快，但包装不够严实，导致产品有轻微损坏。": "差评",
    "安全阀出现了一些问题，让我有点担心。": "差评",
    "服务电话投诉后降价，但仍觉得不太公平，不满意。": "差评",
    "买完降价了，打了400的服务电话投诉后降价到260，真心黑，太恶心了。": "差评"
}
{
    "安全阀在使用时偶尔会出现一些问题，需要进一步改进。": "差评",
    "物流速度非常快，令人惊讶，服务周到。": "好评",
    "安装师傅虽然遇到了一些挑战，但最终还是解决了问题，感到满意。": "好评",
    "产品的性能稳定，一直以来都表现得很可靠。": "好评",
    "价格略高，但总体来说物有所值。": "好评",
    "购买时需要更多的信息和建议，以避免后悔。": "差评",
    "售后服务表现一般，有时响应不及时。": "差评",
    "配件质量较差，导致使用不够顺畅。": "差评"

In [6]:
# 最后，将包含所有评论的 DataFrame 保存为 CSV 文件
csv_path = 'all_reviews.csv'
all_reviews_df.to_csv(csv_path, index=False, encoding='utf8' )

In [7]:
all_reviews_df

,评论,评价
0,安装师傅虽然有黑点，但最终还是解决了问题，感谢他们的努力。,好评
1,物流真的很迅速，让人惊喜。,好评
2,安全阀性能很稳定，非常安心，好评！,好评
3,喷头的质量很不错，一切正常，满意度很高。,好评
4,价格太高了，200的安装费真的令人不满意。,差评
5,与我期望的差距太大，额外购买了不少配件，感觉有点浪费。,差评
6,尽管有些配件需要额外购买，但整体体验还是很好的。,好评
7,购买时应该提供更多的建议，我买小了，现在感到后悔。,差评
8,虽然买小了，售后团队还是提供了一些帮助，感到满意。,好评
9,感觉售后团队对问题处理不够积极，有待改进。,差评
